# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
from sklearn.metrics import hamming_loss, jaccard_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import nltk
import pickle
nltk.download('punkt') 
import sqlite3
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\letsm005\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
# load data from database
#engine = create_engine('sqlite:///InsertDatabaseName.db')
conn = sqlite3.connect('etl_disaster_data.db')

# Read data from SQLite database into a DataFrame
query = "SELECT * FROM etl_disaster_table"
df = pd.read_sql_query(query, conn).head(5000)

# Close the connection
conn.close()
#df = 
#X = 
#Y = 

In [15]:
df = pd.get_dummies(df.drop(["original"],axis=1), columns=["genre"]) 
df.head(2)

,id,category_0,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,...,category_28,category_29,category_30,category_31,category_32,category_33,category_34,category_35,message,genre_direct
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Weather update - a cold front from Cuba that c...,1
1,7,1,0,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,Is the Hurricane over or is it not over,1


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df.drop(["message","id"],axis=1), test_size=0.2, random_state=42
)

In [17]:
# Define pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Text preprocessing
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classifier
])



In [18]:
### 4. Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=30,
                                                                        n_estimators=200)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)
# Calculate accuracy for each label
accuracies = [accuracy_score(y_test[label], y_pred[:, idx]) for idx, label in enumerate(y_test.columns)]
print("Accuracy for each label:", accuracies)

# Calculate Hamming Loss for each label
hamming_losses = [hamming_loss(y_test[label], y_pred[:, idx]) for idx, label in enumerate(y_test.columns)]
print("Hamming Loss for each label:", hamming_losses)

# Calculate Jaccard Score for each label
jaccard_scores = [jaccard_score(y_test[label], y_pred[:, idx], average=None) for idx, label in enumerate(y_test.columns)]
print("Jaccard Score for each label:", jaccard_scores)

# Calculate F1 Score for each label
f1_scores = [f1_score(y_test[label], y_pred[:, idx], average=None) for idx, label in enumerate(y_test.columns)]
print("F1 Score for each label:", f1_scores)


Accuracy for each label: [0.806, 0.815, 0.997, 0.793, 0.921, 0.942, 0.962, 0.976, 0.994, 1.0, 0.969, 0.951, 0.937, 0.991, 0.987, 0.979, 0.982, 0.963, 0.829, 0.956, 0.968, 0.956, 0.994, 0.997, 0.988, 0.997, 0.994, 0.974, 0.915, 0.971, 0.979, 0.995, 0.968, 0.991, 0.974, 0.784, 1.0]
Hamming Loss for each label: [0.194, 0.185, 0.003, 0.207, 0.079, 0.058, 0.038, 0.024, 0.006, 0.0, 0.031, 0.049, 0.063, 0.009, 0.013, 0.021, 0.018, 0.037, 0.171, 0.044, 0.032, 0.044, 0.006, 0.003, 0.012, 0.003, 0.006, 0.026, 0.085, 0.029, 0.021, 0.005, 0.032, 0.009, 0.026, 0.216, 0.0]
Jaccard Score for each label: [array([0.23694779, 0.79383634, 0.        ]), array([0.69967532, 0.67486819]), array([0.997, 0.   ]), array([0.64493997, 0.66826923]), array([0.92052314, 0.07058824]), array([0.94153226, 0.12121212]), array([0.962, 0.   ]), array([0.976, 0.   ]), array([0.994, 0.   ]), array([1.]), array([0.96497175, 0.78767123]), array([0.93828715, 0.80784314]), array([0.93159609, 0.55633803]), array([0.991, 0.   ]),

In [20]:


# Assuming y_test and y_pred are your true labels and predicted labels respectively

# Flatten y_test and y_pred to fit classification_report
y_test_flat = y_test.values.ravel()
y_pred_flat = y_pred.ravel()

# Generate the classification report
report = classification_report(y_test_flat, y_pred_flat)

print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     32245
           1       0.87      0.73      0.79      4751
           2       0.00      0.00      0.00         4

    accuracy                           0.95     37000
   macro avg       0.61      0.57      0.59     37000
weighted avg       0.95      0.95      0.95     37000



### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
param_grid = {
    'tfidf__max_features': [1000, 2000, 3000],  # Number of features to consider
    'tfidf__ngram_range': [(1, 1), (1, 2)],      # Range of n-grams
    'clf__estimator__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'clf__estimator__max_depth': [10, 20, 30],       # Maximum depth of the tree
}

# Perform grid search cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, n_jobs=-1)

# Fit grid search on training data
grid_search.fit(X_train, y_train)

# Evaluate performance on test set
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)

In [ ]:
print(best_pipeline)

In [9]:
# Define pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),  # Text preprocessing
    ('clf', MultiOutputClassifier(RandomForestClassifier(
        max_depth=30,n_estimators=200
    )))  # Multi-output classifier
])

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
### 4. Train pipeline
pipeline.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = pipeline.predict(X_test)
# Calculate accuracy for each label
accuracies = [accuracy_score(y_test[label], y_pred[:, idx]) for idx, label in enumerate(y_test.columns)]
print("Accuracy for each label:", accuracies)

# Calculate Hamming Loss for each label
hamming_losses = [hamming_loss(y_test[label], y_pred[:, idx]) for idx, label in enumerate(y_test.columns)]
print("Hamming Loss for each label:", hamming_losses)

# Calculate Jaccard Score for each label
jaccard_scores = [jaccard_score(y_test[label], y_pred[:, idx], average=None) for idx, label in enumerate(y_test.columns)]
print("Jaccard Score for each label:", jaccard_scores)

# Calculate F1 Score for each label
f1_scores = [f1_score(y_test[label], y_pred[:, idx], average=None) for idx, label in enumerate(y_test.columns)]
print("F1 Score for each label:", f1_scores)


In [ ]:

# Assuming y_test and y_pred are your true labels and predicted labels respectively

# Flatten y_test and y_pred to fit classification_report
y_test_flat = y_test.values.ravel()
y_pred_flat = y_pred.ravel()

# Generate the classification report
report = classification_report(y_test_flat, y_pred_flat)

print("Classification Report:")
print(report)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:

# Assuming y_test and y_pred are your true labels and predicted labels respectively

# Flatten y_test and y_pred to fit classification_report
y_test_flat = y_test.values.ravel()
y_pred_flat = y_pred.ravel()

# Generate the classification report
report = classification_report(y_test_flat, y_pred_flat)

print("Classification Report:")
print(report)


### 9. Export your model as a pickle file

In [ ]:
# Assuming 'pipeline' is your trained pipeline
pipeline.fit(X_train, y_train)  # Train your pipeline if not already done

# Serialize the pipeline using pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.